# Kernals

In [1]:
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, fbeta_score, accuracy_score


from scripts.kernals import *
from scripts.nested_CV import nested_cv

In [2]:
# data
file_path = "../data/laser.mat"
mat = loadmat(file_path)

X = mat["X"]
y = mat["Y"].reshape(200)

print(X.shape, y.shape)

(200, 60) (200,)


In [3]:
# configs
seed = 42  # for repeatability
np.random.seed(42)

fbeta_scorer = make_scorer(fbeta_score, beta=2, pos_label=-1)


In [ ]:
# normalizing the data
X = (X - X.mean(axis=1, keepdims=True)) / X.std(axis=1, keepdims=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed
)

<class 'numpy.ndarray'>


## SVC DTW kernal test

In [5]:
from scripts.sk_kernals import dtw_SVC

clf_test = dtw_SVC(estimate=False)

clf_test.fit(X_train, y_train)
print(clf_test.dist_mat_)


[[0.         5.05799569 4.46987422 ... 4.01389087 3.43100639 4.47934648]
 [5.05799569 0.         4.21381952 ... 3.80935114 3.68925622 3.94933136]
 [4.46987422 4.21381952 0.         ... 2.74945927 3.12349513 4.50894697]
 ...
 [4.01389087 3.80935114 2.74945927 ... 0.         2.74829099 4.2536341 ]
 [3.43100639 3.68925622 3.12349513 ... 2.74829099 0.         4.09370036]
 [4.47934648 3.94933136 4.50894697 ... 4.2536341  4.09370036 0.        ]]


In [6]:
predictions = clf_test.predict(X_test)

print(classification_report(y_test, predictions))

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
clf_test2 = KernelEstimator(k2_hyp, .01, 1)
clf_test2.fit(X_train, y_train)

KernelEstimator(k=<function <listcomp>.<lambda> at 0x0000013B0C36BEC0>,
                k_param=0.01)

In [ ]:
predictions2 = clf_test2.predict(X_test)

print(classification_report(y_test, predictions2))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        17
           1       0.57      1.00      0.73        23

    accuracy                           0.57        40
   macro avg       0.29      0.50      0.37        40
weighted avg       0.33      0.57      0.42        40



c:\Users\haani\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\haani\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\haani\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

# Support Vector Machines

Classifiers, that use the hinge loss and use the L2 regularizer

In the dual mode, they can be used in conjunction with kernal methods, which is what is employed in the following notebook.


## 1. SVC with Polynomial Kernal

### Description
The idea here is to map the data into a higher dimensional space, such that data is linearly seperable in that space and we can then perform linear classification


### Nested CV
We run a nested cross validation function for the following hyperparameters of the Polynomial Kernal:
- p - degree of polynomial
- lambda - regularization coefficient

In [5]:
param_grid_poly = {
    'k_param' : [2,3,4,5,6], # degree of polynomial
    'lbda' : [1,10,100],
    'eta' : [1, .5, .1, .01, .001]
}
results = nested_cv(KernelEstimator(k_poly, eta=.01), X, y, param_grid=param_grid_poly, scoring=fbeta_scorer, random_state=seed)

print(">>>>RESULTS<<<<")
print("Mean f2 score (of negative class) over 5 by 5 rounds of Nested CV ", results["mean_score"])
print("Best parameters ", results["star_params"])

>>>>RESULTS<<<<
Mean f2 score (of negative class) over 5 by 5 rounds of Nested CV  0.8263305322128852
Best parameters  {'eta': 1, 'k_param': 2, 'lbda': 10}


In [6]:
print(results)

{'mean_score': 0.8263305322128852, 'star_params': {'eta': 1, 'k_param': 2, 'lbda': 10}, 'outer_scores': [0.8333333333333334, 0.7983193277310925, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334], 'std_score': 0.014005602240896354, 'best_params': [{'eta': 1, 'k_param': 2, 'lbda': 10}, {'eta': 1, 'k_param': 3, 'lbda': 100}, {'eta': 1, 'k_param': 2, 'lbda': 10}, {'eta': 1, 'k_param': 2, 'lbda': 10}, {'eta': 1, 'k_param': 2, 'lbda': 100}]}


In [7]:
clf_poly = KernelEstimator(k_poly, 2, 10, eta=1)
clf_poly.fit(np.delete(X, [35, 162], axis=0), np.delete(y, [35,162], axis=0))

odd_laser_faulty = clf_poly.predict([X[35]])
odd_laser_non_faulty = clf_poly.predict([X[162]])
print(odd_laser_faulty, odd_laser_non_faulty)

[1.] [-1.]


In [8]:
y_preds_poly = clf_poly.predict(X)
print(fbeta_score(y, y_preds_poly, beta=2, pos_label=-1))
print(classification_report(y, y_preds_poly))

0.801056338028169
              precision    recall  f1-score   support

          -1       0.54      0.91      0.68       100
           1       0.72      0.23      0.35       100

    accuracy                           0.57       200
   macro avg       0.63      0.57      0.51       200
weighted avg       0.63      0.57      0.51       200



## 2. SVC with RBF

### Nested CV
We run a nested cross validation function for the following hyperparameters of the Polynomial Kernal:
- gamma - scaling of the RBF function
- lambda - regularization coefficient


In [9]:
param_grid_rbf = {
    'k_param' : [.001,.01,.1,1,10,100], # gamma
    'lbda' : [1,10,100],
    'eta' : [1, .5, .01, .001]
}
results = nested_cv(KernelEstimator(k_rbf, eta=.01), X, y, param_grid=param_grid_rbf, scoring=fbeta_scorer, random_state=seed)

print(">>>>RESULTS<<<<")
print("Mean f2 score (of negative class) over 5 by 5 rounds of Nested CV ", results["mean_score"])
print("Best parameters ", results["star_params"])

>>>>RESULTS<<<<
Mean f2 score (of negative class) over 5 by 5 rounds of Nested CV  0.82600553153684
Best parameters  {'eta': 0.001, 'k_param': 10, 'lbda': 10}


In [10]:
clf_rbf = KernelEstimator(k_rbf, 10, 10, eta=.001)
clf_rbf.fit(np.delete(X, [35, 162], axis=0), np.delete(y, [35,162], axis=0))

odd_laser_faulty = clf_rbf.predict([X[35]])
odd_laser_non_faulty = clf_rbf.predict([X[162]])
print(odd_laser_faulty, odd_laser_non_faulty)

[1.] [1.]


In [11]:
y_preds_rbf = clf_rbf.predict(X)
print(classification_report(y,y_preds_rbf))

              precision    recall  f1-score   support

          -1       0.57      0.74      0.64       100
           1       0.63      0.44      0.52       100

    accuracy                           0.59       200
   macro avg       0.60      0.59      0.58       200
weighted avg       0.60      0.59      0.58       200



## 3. SVC with DTW

### Nested CV
We run a nested cross validation function for the following hyperparameters of the Polynomial Kernal:
- gamma - smoothing of the DTW function
- (maybe) k - distance (Euclidean, manhattan, )
- lambda - regularization coefficient

In [72]:
param_grid_dtw = {
    'k_param' : [.01, .1,1], # gamma
    'lbda' : [1, 10, 100],
}
results = nested_cv(KernelEstimator(k2_hyp, eta=.01), X, y, param_grid=param_grid_dtw, scoring=fbeta_scorer, random_state=seed, inner_cv=3, outer_cv=3)

print(">>>>RESULTS<<<<")
print("Mean f2 score (of negative class) over 3 by 3 rounds of Nested CV ", results["mean_score"])
print("Best parameters ", results["star_params"])

>>>>RESULTS<<<<
Mean f2 score (of negative class) over 3 by 3 rounds of Nested CV  0.8333058285406233
Best parameters  {'k_param': 0.01, 'lbda': 1}


In [73]:
print(results)

{'mean_score': 0.8333058285406233, 'star_params': {'k_param': 0.01, 'lbda': 1}, 'outer_scores': [0.8374384236453202, 0.8291457286432161, 0.8333333333333334], 'std_score': 0.0033855344220863094, 'best_params': [{'k_param': 0.01, 'lbda': 1}, {'k_param': 0.01, 'lbda': 1}, {'k_param': 0.01, 'lbda': 1}]}


In [109]:
clf_dtw = KernelEstimator(k2_hyp, .01, 1, eta=.01)
clf_dtw.fit(np.delete(X, [35, 162], axis=0), np.delete(y, [35,162], axis=0))

odd_laser_faulty = clf_dtw.predict([X[35]])
odd_laser_non_faulty = clf_dtw.predict([X[162]])
print(odd_laser_faulty, odd_laser_non_faulty)

[-1.] [-1.]


In [110]:
y_preds_dtw = clf_dtw.predict(X)
print(classification_report(y,y_preds_dtw))

print(y_preds_dtw)

              precision    recall  f1-score   support

          -1       0.50      0.99      0.66       100
           1       0.50      0.01      0.02       100

    accuracy                           0.50       200
   macro avg       0.50      0.50      0.34       200
weighted avg       0.50      0.50      0.34       200

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1.

In [111]:
print(fbeta_score(y, y_preds_dtw, beta=2 , pos_label=-1))
print(accuracy_score(y, y_preds_dtw))

0.8277591973244147
0.5
